In [1]:
!pip install pennylane
!pip install noisyopt
!pip install ipympl
!pip install ibm_quantum_widgets

In [2]:
import numpy as np

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.providers.aer import QasmSimulator

from pennylane import qchem
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from noisyopt import minimizeSPSA
import time

# Loading your IBM Quantum account(s)
# provider = IBMQ.load_account()

In [3]:
# The number of iterations of the optimizer
niter_spsa = 200

# The true energy of the H2 molecule
true_energy = -1.136189454088

# Seed number, for reproducibility of random initialization
seednum=0

# Optimizer hyperparameters
# TODO : search the documentation of minimizeSPSA to see
# what they mean and are used for.
c_value=0.3
a_value=1.5

# Setup the data of the molecule to optimize for
symbols = ["H", "H"]
coordinates = np.array([0.0, 0.0, -0.6614, 0.0, 0.0, 0.6614])

# TODO: You need a function from quantum chemistry (qchem) that,
# using the symbols and chemistry coordinates of the H2 molecule,
# gives you the corresponding hamiltonian
h2_ham, num_qubits = qchem.molecular_hamiltonian(symbols, coordinates)

# Variational ansatz for H2 
def circuit(params, wires): 
    qml.BasisState(np.array([1, 1, 0, 0]), wires=wires)
    for i in wires:
        qml.Rot(*params[i], wires=i)
            
    # TODO: add 3 controlled-NOT gates for entanglement, 
    # where the control - target pairs are [(2, 3), (2, 0), (3, 1)]
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[2, 0])
    qml.CNOT(wires=[3, 1])

num_params=3 

# TODO : find the name of the pennylane qubit device (the default one)
devicename= 

# TODO: how do you instantiate a quantum device on pennylane?
# You should find a function that uses the devicename and number of 
# qubits, also known as wires
dev =  

def exp_val_circuit(params):
    circuit(params, range(dev.num_wires))
    # TODO: measure and return the expectation value of the circuit
    # with respect to the Hamiltonian of the H2 circuit measured above
    return 

# Pennylane uses Quantum Nodes (Qnodes)
# TODO: create a QNode corresponding to the
# last circuit defined above and that acts on the device we created
cost_spsa = 

# Wrapping the cost function and flattening the parameters to be compatible with noisyopt which assumes a flat array of input parameters
def wrapped_cost(params):  
    return cost_spsa(params.reshape(num_qubits, num_params))


## Here we define the callback function that is used by the SPSA optimizer
def callback_fn(xk):
    cost_val = wrapped_cost(xk)
    h2_spsa_energies.append(cost_val)
    
    # We have evaluated every term twice, so we need to make up for this
    num_executions = int(dev.num_executions / 2) 
    h2_spsa_device_executions.append(num_executions)
    h2_spsa_params.append(xk)


results=[]

## Set initial rotation parameters
np.random.seed(seednum)
init_params = np.random.normal(0, np.pi, (num_qubits, 3), requires_grad=True)
initial_name='random seed '+str(seednum)
params = init_params.copy().reshape(num_qubits * num_params) #flatten

h2_spsa_device_executions = [0]
h2_spsa_energies = [wrapped_cost(params)]
h2_spsa_params = []

## TO DO : add the parameters to the minimizeSPSA function
res = minimizeSPSA()

print("Result:", res)
results.append(h2_spsa_device_executions)
results.append(h2_spsa_energies)
results.append(h2_spsa_params)
print()
print(f"Final estimated value of the ground-state energy = {h2_spsa_energies [-1]:.8f} Ha")
print(
   f"Accuracy with respect to the true energy: {np.abs(h2_spsa_energies [-1] - true_energy):.8f} Ha"
)

SyntaxError: invalid syntax (3853681035.py, line 40)

In [ ]:
## Energy vs. iteration plot
%matplotlib ipympl 
plt.figure(figsize=(10, 6))
plt.plot(range(niter_spsa + 1), results[1], label = "Energy level of the current VQE using SPSA")
plt.title("H2 energy from the VQE using SPSA using "+ devicename, fontsize=16)
plt.xlabel("Number of optimization iterations", fontsize=14)
plt.ylabel("Energy (Ha)", fontsize=14)
plt.grid()
plt.hlines(true_energy, 0, niter_spsa+2, label="True energy level of H2", color = "red")
plt.legend(fontsize=14)
plt.show()